# Train uBERTa

Here, we'll use train the `distilBERT` model on the sequences from the prepared base dataset.

## Prerequisites

This notebook requires:
- [DS_BASE](https://drive.google.com/file/d/15fQP5ldYNvV1YY2T2Qza9CNFdYj4zZg8/view?usp=sharing) (`../data/DS_BASE_4.7_seqs.tsv`)
- [dataset_labeling](https://drive.google.com/file/d/1-R1zLJRrJg3KXAaqDe9T60s9MXdrv4RO/view?usp=sharing) (`../data/dataset_labeling_v4.7.tsv`)

One can either download or obtain the requirements manually via running `prepare_base_dataset_v4.7.ipynb`.

For instance, to download the data, starting from the project's root:
```bash
gdown --fuzzy https://drive.google.com/file/d/15fQP5ldYNvV1YY2T2Qza9CNFdYj4zZg8/view?usp=sharing
gdown --fuzzy https://drive.google.com/file/d/1-R1zLJRrJg3KXAaqDe9T60s9MXdrv4RO/view?usp=sharing
tar -xzf DS_BASE_4.7_seqs.tsv.tar.gz
tar -xzf dataset_labeling_4.7.tsv.tar.gz
```

## Setup

In [ ]:
import logging
from collections import Counter
from pathlib import Path

import pandas as pd
import pytorch_lightning as pl
import torch
from sklearn.utils.class_weight import compute_class_weight
from transformers import DistilBertConfig, get_linear_schedule_with_warmup

from uBERTa.base import VALID_START
from uBERTa.loader import uBERTaLoader
from uBERTa.model import uBERTa_classifier, WeightedDistilBertClassifier2
from uBERTa.tokenizer import DNATokenizer
from uBERTa.utils import split_values

In [ ]:
def load_existing(paths):
    loader = (
        lambda p: None if not p.exists() else 
        (pd.read_hdf(p) if p.suffix == '.h5' else torch.load(p))
    )
    return {k: loader(v) for k, v in paths.items()}

def parse_base(path_base, path_labels, min_seq_size, max_seq_size):
    df = pd.read_csv(path_base, sep='\t')
    df['SeqSize'] = df['Seq'].apply(len)
    print(f'Initial ds: {len(df)}')
    df = df.merge(pd.read_csv(path_labels, sep='\t'), on='GeneID')
    print(f'Labeled genes: {len(df)}')
    df = df[(df.SeqSize >= min_seq_size) & (df.SeqSize <= max_seq_size)]
    print(f'Conforming to size threshold: {len(df)}')
    split_values(df, 'SeqEnum')
    split_values(df, 'Classes')
    split_values(df, 'Signal', dtype=float)
    return df

def calc_scheduler_steps(loader, warmup_perc=0.1, max_epochs=100):
    epoch_steps = len(loader.train_dataloader())
    total_steps = epoch_steps * max_epochs
    warmup_steps = int(warmup_perc * total_steps)
    return warmup_steps, total_steps

In [ ]:
KMER = 3
MIN_SEQ_SIZE = 30
MAX_SEQ_SIZE = 3000
WINDOW = 100
STEP = WINDOW // 5
MAX_EPOCHS = 300

# Valid start codons
# STARTS = ('ACG', 'ATC', 'ATG', 'ATT', 'CTG', 'GTG')
STARTS = VALID_START
# Name of the dataset
DS = f'ws{WINDOW}_step{STEP}_{"_".join(STARTS)}'
# Name of the model
MODEL = f'{DS}_nopretrain_tokenlevel_signal'

MODEL_PATH = Path(f'../models/{MODEL}')
# PRETRAINED_PATH = Path('../models/ws100_step50_pretrain/')
PATH_BASE_DS = Path('../data/DS_BASE_v4.7_seqs.tsv')
PATH_LABELS = Path('../data/dataset_labeling_v4.7.tsv')

datasets_base = Path(f'../data/datasets/{DS}')
datasets_base.mkdir(exist_ok=True, parents=True)

DATASETS = {
    'train_ds': datasets_base / 'train_ds.h5',
    'val_ds': datasets_base / 'val_ds.h5',
    'test_ds': datasets_base / 'test_ds.h5',
    'train_tds': datasets_base / 'train_tds.bin',
    'val_tds': datasets_base / 'val_tds.bin',
    'test_tds': datasets_base / 'test_tds.bin'
}

In [4]:
logging.basicConfig(level=logging.DEBUG)

## Prepare datasets
Preparing datasets is time-consuming. The code below will check if the required datasets are present in `datasets_base`: if not (e.g., first time running the notebook), the process will start from the base dataset. The data preparation is fully encapsulated into the `setup` method of `uBERTaLoader`. Basically, this will kmerize sequence data, compose token labels, aggregate the ribo-seq signal for each token, and use the sliding window on sequences to unify the input size.

In [5]:
ds_paths = (DATASETS['train_ds'], DATASETS['val_ds'], DATASETS['test_ds'])
if any(not p.exists() for p in ds_paths):
    ds = parse_base(PATH_BASE_DS, PATH_LABELS, MIN_SEQ_SIZE, MAX_SEQ_SIZE)
else:
    ds = None

In [6]:
tokenizer = DNATokenizer(kmer=KMER)
loader = uBERTaLoader(
    ds, WINDOW, STEP, tokenizer, 
    **load_existing(DATASETS),
    scale_signal_bounds=(0.0, 10.0),
    is_mlm_task=False,
    valid_start_codons=STARTS,
    batch_size=2 ** 10)

In [7]:
loader.setup()
loader.save_all(datasets_base)

DEBUG:uBERTa.loader:Skipping existing train_ds.h5
DEBUG:uBERTa.loader:Skipping existing val_ds.h5
DEBUG:uBERTa.loader:Skipping existing test_ds.h5
DEBUG:uBERTa.loader:Skipping existing train_tds.bin
DEBUG:uBERTa.loader:Skipping existing val_tds.bin
DEBUG:uBERTa.loader:Skipping existing test_tds.bin


Below we'll count unmasked tokens (defined by `STARTS` above) and count positive and negative classes. The `weight` can then be used with `uBERTa_classifier`, although it doesn't lead to performance gains and will likely require manually adjusting the threshold for converting the predictions into binary labels for the desired balance between recall and precision.

In [8]:
for tds in [loader.val_tds, loader.test_tds, loader.train_tds]:
    classes, inp_ids = tds.tensors[2], tds.tensors[0]
    mask = classes != -100
    mask0 = classes == 0
    mask1 = classes == 1
    weight = compute_class_weight('balanced', classes=[0, 1], y=classes[mask].numpy())
    codons = inp_ids[mask]
    print(Counter(codons.numpy()), weight, mask0.sum(), mask1.sum())

weight = (1, 5)

Counter({44: 43728, 20: 35838, 8: 28844, 60: 28225, 28: 24327, 10: 17956, 12: 16524, 11: 15357, 9: 11097, 16: 9913}) [ 0.51478792 17.40569155] tensor(225150) tensor(6659)
Counter({44: 39064, 20: 31430, 8: 26533, 60: 26436, 28: 19350, 10: 15097, 12: 13986, 11: 12423, 9: 9165, 16: 9058}) [ 0.51525867 16.88412804] tensor(196544) tensor(5998)
Counter({44: 377307, 20: 320077, 8: 247527, 60: 241475, 28: 200640, 10: 153610, 11: 132370, 12: 131539, 16: 92178, 9: 86258}) [ 0.51405832 18.28306288] tensor(1928751) tensor(54230)


In [9]:
sizes = list(map(len, [loader.val_tds, loader.test_tds, loader.train_tds]))
sizes, sum(sizes)

([16582, 14755, 145119], 176456)

## Setup model

In [10]:
warmup_steps, total_steps = calc_scheduler_steps(
    loader, warmup_perc=0.01, max_epochs=MAX_EPOCHS)
warmup_steps, total_steps

(426, 42600)

We'll use the config of the pretrained model and add one field so that the model uses the experimental signal internally.

In [11]:
config = DistilBertConfig(
    dim=36, hidden_dim=256, n_heads=3, n_layers=3
)
config.vocab_size = tokenizer.vocab_size
config.use_signal = True
config

DistilBertConfig {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 36,
  "dropout": 0.1,
  "hidden_dim": 256,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 3,
  "n_layers": 3,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "transformers_version": "4.16.2",
  "use_signal": true,
  "vocab_size": 69
}

In [12]:
# config = DistilBertConfig.from_pretrained(PRETRAINED_PATH)
# config.use_signal = True

Note: manually providing the `device` argument is needed only if using `weight`. It will also restrict the model to the provided device, and using distributed training may cause an exception. I couldn't (yet) find a way to use `weight` with distributed training.

In [13]:
model = uBERTa_classifier(
    model=WeightedDistilBertClassifier2,
    config=config,
    opt_kwargs={'lr': 1e-5, 'weight_decay': 0.1, 'eps': 1e-8},
    scheduler=get_linear_schedule_with_warmup,
    scheduler_kwargs={'num_warmup_steps': warmup_steps, 'num_training_steps': total_steps},
    weight=weight, device='cuda:1'
)
pl.utilities.model_summary.summarize(model)
# model.summarize()
# It's a bit nested, most certainly unnecessarily
# model.model.bert = model.model.bert.from_pretrained(PRETRAINED_PATH)
# model.model.config.use_signal = True

  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | WeightedDistilBertClassifier2 | 96.1 K
--------------------------------------------------------
96.1 K    Trainable params
0         Non-trainable params
96.1 K    Total params
0.385     Total estimated model params size (MB)

## Train the model

In [14]:
stopper = pl.callbacks.early_stopping.EarlyStopping(
    monitor='val_loss', 
    verbose=True, mode='min', 
    min_delta=1e-6,
    patience=20)
pointer = pl.callbacks.ModelCheckpoint(
    monitor='val_loss', 
    dirpath=f'../models/checkpoints/{MODEL}', 
    verbose=True, mode='min')
logger = pl.loggers.TensorBoardLogger('../logs', f'{MODEL}')
lr_monitor = pl.callbacks.LearningRateMonitor('epoch')
bar = pl.callbacks.TQDMProgressBar()

In [15]:
gpus = [0]
trainer = pl.Trainer(
    gradient_clip_val=1.0,
    stochastic_weight_avg=True,
    accelerator="gpu",
    precision=16,
    gpus=gpus,
    callbacks=[stopper, pointer, bar, lr_monitor],
    logger=logger,
    max_epochs=MAX_EPOCHS
)

Using 16bit native Automatic Mixed Precision (AMP)
/home/ivan/miniconda3/envs/uberta/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:58: LightningDeprecationWarning: Setting `Trainer(stochastic_weight_avg=True)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.stochastic_weight_avg.StochasticWeightAveraging` directly to the Trainer's `callbacks` argument instead.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [16]:
trainer.fit(model, loader)

/home/ivan/miniconda3/envs/uberta/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | WeightedDistilBertClassifier2 | 96.1 K
--------------------------------------------------------
96.1 K    Trainable params
0         Non-trainable params
96.1 K    Total params
0.192     Total estimated model params size (MB)
/home/ivan/miniconda3/envs/uberta/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /home/ivan/code/uBERTa/models/checkpoints/ws100_step20_AAG_ACG_AGG_ATA_ATC_ATG_ATT_CTG_GTG_TTG_nopretrain_tokenlevel_signal exists and is not empty.
  rank_zero_

Validation sanity check: 0it [00:00, ?it/s]

/home/ivan/miniconda3/envs/uberta/lib/python3.8/site-packages/deprecate/deprecation.py:115: FutureWarning: The `f1` was deprecated since v0.7 in favor of `torchmetrics.functional.classification.f_beta.f1_score`. It will be removed in v0.8.
  stream(template_mgs % msg_args)


Training: 0it [00:00, ?it/s]

/home/ivan/miniconda3/envs/uberta/lib/python3.8/site-packages/pytorch_lightning/loops/optimization/closure.py:35: LightningDeprecationWarning: One of the returned values {'logits', 'outputs'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  rank_zero_deprecation(


...

Epoch 214, global step 30529: val_loss was not in top 1


Validating: 0it [00:00, ?it/s]

Metric val_loss improved by 0.000 >= min_delta = 1e-06. New best score: 0.106
Epoch 215, global step 30671: val_loss reached 0.10599 (best 0.10599), saving model to "/home/ivan/code/uBERTa/models/checkpoints/ws100_step20_AAG_ACG_AGG_ATA_ATC_ATG_ATT_CTG_GTG_TTG_nopretrain_tokenlevel_signal/epoch=215-step=30671.ckpt" as top 1


Validating: 0it [00:00, ?it/s]

Epoch 216, global step 30813: val_loss was not in top 1
/home/ivan/miniconda3/envs/uberta/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


## Save the model.

Check the checkpoints directory (`../models/checkpoints/{MODEL}`) and put a name of the desired checkpoint file below.

In [21]:
# ckpt = "epoch=3-step=919.ckpt"
# _model = uBERTa_classifier.load_from_checkpoint(
#     f'../models/checkpoints/{MODEL}/{ckpt}', 
#     model=WeightedDistilBertClassifier2,
#     config=config)
# _model.model.save_pretrained(MODEL_PATH)

- In our case, the model has converged without early stopping, so we save the trained model without the checkpoint restoration

In [17]:
model.model.save_pretrained(MODEL_PATH)

In [18]:
MODEL_PATH

PosixPath('../models/ws100_step20_AAG_ACG_AGG_ATA_ATC_ATG_ATT_CTG_GTG_TTG_nopretrain_tokenlevel_signal')